# spam detection with natural language processing 

In [1]:
import numpy as np   
import pandas as pd  

# 1) LOADING DATA SET 

In [2]:
dt = pd.read_csv("SPAM-210331-134237 (1).csv", encoding= 'unicode_escape')
dt.head(10)

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
dt['spam'] = dt['type'].map( {'spam': 1, 'ham': 0} ).astype(int)
dt.head(5)

,type,text,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [4]:
print("COLUMNS IN THE GIVEN DATA:")
for col in dt.columns: 
    print(col) 

COLUMNS IN THE GIVEN DATA:
type
text
spam


In [5]:
t=len(dt['type'])
print("NO OF ROWS IN REVIEW COLUMN:",t)
t=len(dt['text'])
print("NO OF ROWS IN liked COLUMN:",t)

NO OF ROWS IN REVIEW COLUMN: 116
NO OF ROWS IN liked COLUMN: 116


# 2)Tokenization

In [6]:
dt['text'][1]#before   

'Ok lar... Joking wif u oni...'

In [7]:
def tokenizer(text):
    return text.split()

In [8]:
dt['text']=dt['text'].apply(tokenizer)

In [9]:
dt['text'][1]#after 

['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...']

# 3) STEMMING 

In [10]:
dt['text'][1]#before 

['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...']

In [11]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer("english", ignore_stopwords=False)

In [12]:
def stem_it(text):
    return [porter.stem(word) for word in text]

In [13]:
dt['text']=dt['text'].apply(stem_it)

In [14]:
dt['text'][1] #after stemming 

['ok', 'lar...', 'joke', 'wif', 'u', 'oni...']

# 4) LEMMITIZATION 

In [15]:
dt['text'][15] #before

['xxxmobilemovieclub:',
 'to',
 'use',
 'your',
 'credit,',
 'click',
 'the',
 'wap',
 'link',
 'in',
 'the',
 'next',
 'txt',
 'messag',
 'or',
 'click',
 'here>>',
 'http://wap.',
 'xxxmobilemovieclub.com?n=qjkgighjjgcbl']

In [16]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

In [17]:
def lemmit_it(text):
    return [lemmatizer.lemmatize(word, pos ="a") for word in text]

In [18]:
dt['text']=dt['text'].apply(lemmit_it)

In [19]:
dt['text'][15] #after

['xxxmobilemovieclub:',
 'to',
 'use',
 'your',
 'credit,',
 'click',
 'the',
 'wap',
 'link',
 'in',
 'the',
 'next',
 'txt',
 'messag',
 'or',
 'click',
 'here>>',
 'http://wap.',
 'xxxmobilemovieclub.com?n=qjkgighjjgcbl']

# 5)STOPWORD REMOVAL

In [20]:
dt['text'][21] #before 

['iâ\x80\x98m',
 'go',
 'to',
 'tri',
 'for',
 '2',
 'month',
 'ha',
 'ha',
 'onli',
 'joke']

In [21]:
#from nltk.corpus import stopwords
#print(stopwords.words('english'))
#set(['a','t','d','y'])

In [22]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [23]:
def stop_it(text):
    review = [word for word in text if not word in stop_words ] 
    return review

In [24]:
dt['text']=dt['text'].apply(stop_it)

In [25]:
dt['text'][21] #after 

['iâ\x80\x98m', 'go', 'tri', '2', 'month', 'ha', 'ha', 'onli', 'joke']

In [26]:
dt.head(10)

,type,text,spam
0,ham,"[go, jurong, point,, crazy.., avail, onli, bug...",0
1,ham,"[ok, lar..., joke, wif, u, oni...]",0
2,spam,"[free, entri, 2, wkli, comp, win, fa, cup, fin...",1
3,ham,"[u, dun, say, earli, hor..., u, c, alreadi, sa...",0
4,ham,"[nah, think, goe, usf,, live, around, though]",0
5,spam,"[freemsg, hey, darl, 3, week, word, back!, i'd...",1
6,ham,"[even, brother, like, speak, me., treat, like,...",0
7,ham,"[per, request, mell, mell, (oru, minnaminungin...",0
8,spam,"[winner!!, valu, network, custom, select, rece...",1
9,spam,"[mobil, 11, month, more?, u, r, entitl, updat,...",1


In [27]:
dt['text']=dt['text'].apply(' '.join)

In [28]:
dt.head()

,type,text,spam
0,ham,"go jurong point, crazy.. avail onli bugi n gre...",0
1,ham,ok lar... joke wif u oni...,0
2,spam,free entri 2 wkli comp win fa cup final tkts 2...,1
3,ham,u dun say earli hor... u c alreadi say...,0
4,ham,"nah think goe usf, live around though",0


#  6) Transform Text Data into TDF /TF-IDF Vectors   

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
y=dt.spam.values
x=tfidf.fit_transform(dt['text'])

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_text,y_train,y_text=train_test_split(x,y,random_state=1,test_size=0.2,shuffle=False)

#  7) Classification using Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_text)
from sklearn.metrics import accuracy_score
acc_log = accuracy_score(y_pred, y_text)*100
print("accuracy:",acc_log )

accuracy: 87.5


#   8) Classification using LinearSVC Accuracy 

In [32]:
from sklearn.svm import LinearSVC

linear_svc = LinearSVC(random_state=0)
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_text)
acc_linear_svc =accuracy_score(y_pred, y_text) * 100
print("accuracy:",acc_linear_svc)

accuracy: 87.5
